hopa1029, klfl1012  
__________________________________________________________________________________________________________________________________________________________________________________________________________________

# Übungsblatt 2 - Personalabteilung

**DSCB310 – Datenanalyse und Business Intelligence 1**  

Wintersemester 22/23

##### Gliederung des Notebooks:

1. Ziel der Personalabteilung
   
2. Fragestellungen:

   - P1: Prüfen Sie, ob ein Zusammenhang zwischen dem Trinkgeldgeben und dem Vorhandensein eines der folgenden Produkte in einer Bestellung existiert: 24852, 24964, 2120
  
   - P2: Lassen sich regionale Unterschiede im Trinkgeldverhalten erkennen?
  
   - P3: Welche Attribute einer Bestellung wirken sich auf das Trinkgeldverhalten aus?
  
   - P4: Spielt die Vergangenheit eines Users eine Rolle in Hinblick auf die Trinkgeldwahrscheinlichkeit, oder kommt es nur auf Inhalt und Parameter der aktuellen Bestellung an?
  
3. Zusätzliche Fragestellungen

##### Import des Datensatz: 

**Imports:**

In [4]:
import pandas as pd
import plotly.express as px

**Datensatz:**

-- erklärung der einzelnen attribute --

In [ ]:
df = pd.read_parquet("DSCB310 - UE2 - Shopping Carts.parquet")

In [ ]:
df.head()

## 1. Ziel der Personalabteilung:

## 2. Fragestellungen:

#### **P1:** Prüfen Sie, ob ein Zusammenhang zwischen dem Trinkgeldgeben und dem Vorhandensein eines der folgenden Produkte in einer Bestellung existiert: 24852, 24964, 2120:

#### **P2:** Lassen sich regionale Unterschiede im Trinkgeldverhalten erkennen?

#### **P3:** Welche Attribute einer Bestellung wirken sich auf das Trinkgeldverhalten aus?

#### **P4:** Spielt die Vergangenheit eines Users eine Rolle in Hinblick auf die Trinkgeldwahrscheinlichkeit, oder kommt es nur auf Inhalt und Parameter der aktuellen Bestellung an?

## 3. Zusätzliche Fragestellungen: